# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/vinit/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vinit/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/vinit/Desktop/AIM/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/vinit/Desktop/AIM/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/vinit/Desktop/AIM/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '17b446'. Skipping!
Property 'summary' already exists in node '90e0b3'. Skipping!
Property 'summary' already exists in node '255a05'. Skipping!
Property 'summary' already exists in node '13979c'. Skipping!
Property 'summary' already exists in node '46ad6b'. Skipping!
Property 'summary' already exists in node 'db0b30'. Skipping!
Property 'summary' already exists in node '418072'. Skipping!
Property 'summary' already exists in node 'b7c9fb'. Skipping!
Property 'summary' already exists in node '27de7c'. Skipping!
Property 'summary' already exists in node '962eb0'. Skipping!
Property 'summary' already exists in node 'c46008'. Skipping!
Property 'summary' already exists in node '63d787'. Skipping!
Property 'summary' already exists in node '676411'. Skipping!
Property 'summary' already exists in node 'ac22db'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '17b446'. Skipping!
Property 'summary_embedding' already exists in node 'db0b30'. Skipping!
Property 'summary_embedding' already exists in node '46ad6b'. Skipping!
Property 'summary_embedding' already exists in node '418072'. Skipping!
Property 'summary_embedding' already exists in node '255a05'. Skipping!
Property 'summary_embedding' already exists in node '13979c'. Skipping!
Property 'summary_embedding' already exists in node '90e0b3'. Skipping!
Property 'summary_embedding' already exists in node 'b7c9fb'. Skipping!
Property 'summary_embedding' already exists in node '27de7c'. Skipping!
Property 'summary_embedding' already exists in node 'c46008'. Skipping!
Property 'summary_embedding' already exists in node '962eb0'. Skipping!
Property 'summary_embedding' already exists in node '676411'. Skipping!
Property 'summary_embedding' already exists in node '63d787'. Skipping!
Property 'summary_embedding' already exists in node 'ac22db'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is Chapter 2 about in the academic program?,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not include specific...,single_hop_specifc_query_synthesizer
1,What is the regulatoy citation 34 CFR 668.3(a)...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,Volume 8 is it like a standard term or not?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,What is the significance of Title IV in relati...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,What information is provided in Appendix A reg...,[both the credit or clock hours and the weeks ...,Appendix A illustrates the principles related ...,single_hop_specifc_query_synthesizer
5,How do the disbursement timing requirements fo...,[<1-hop>\n\nboth the credit or clock hours and...,Disbursement timing for Pell Grants and TEACH ...,multi_hop_abstract_query_synthesizer
6,What are the program-specific academic year mi...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","For Title IV eligibility, each eligible progra...",multi_hop_abstract_query_synthesizer
7,How do participation requirements for practicu...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Participation and requirements for practicum o...,multi_hop_abstract_query_synthesizer
8,How do Volume 8 and Volume 7 relate to the inc...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 discusses the inclusion of clinical w...,multi_hop_specific_query_synthesizer
9,How do Chapters 2 and 3 collectively address t...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Chapter 2 discusses the inclusion of clinical ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '6fff63'. Skipping!
Property 'summary' already exists in node 'd85ae4'. Skipping!
Property 'summary' already exists in node 'ec3f61'. Skipping!
Property 'summary' already exists in node '1ba269'. Skipping!
Property 'summary' already exists in node '00e096'. Skipping!
Property 'summary' already exists in node 'fe7d6a'. Skipping!
Property 'summary' already exists in node 'f8ae6c'. Skipping!
Property 'summary' already exists in node '097290'. Skipping!
Property 'summary' already exists in node '5a09d4'. Skipping!
Property 'summary' already exists in node 'd74573'. Skipping!
Property 'summary' already exists in node '5b6fb2'. Skipping!
Property 'summary' already exists in node 'c96838'. Skipping!
Property 'summary' already exists in node 'b9bc1f'. Skipping!
Property 'summary' already exists in node 'aae6c3'. Skipping!


...

Property 'summary_embedding' already exists in node '6fff63'. Skipping!
Property 'summary_embedding' already exists in node 'fe7d6a'. Skipping!
Property 'summary_embedding' already exists in node 'c96838'. Skipping!
Property 'summary_embedding' already exists in node 'd74573'. Skipping!
Property 'summary_embedding' already exists in node 'd85ae4'. Skipping!
Property 'summary_embedding' already exists in node 'ec3f61'. Skipping!
Property 'summary_embedding' already exists in node '1ba269'. Skipping!
Property 'summary_embedding' already exists in node 'f8ae6c'. Skipping!
Property 'summary_embedding' already exists in node '5a09d4'. Skipping!
Property 'summary_embedding' already exists in node 'b9bc1f'. Skipping!
Property 'summary_embedding' already exists in node '5b6fb2'. Skipping!
Property 'summary_embedding' already exists in node '097290'. Skipping!
Property 'summary_embedding' already exists in node 'aae6c3'. Skipping!
Property 'summary_embedding' already exists in node '00e096'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Volume 2 about in the context of acade...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide specific informat...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) mean in terms of wee...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What information does Volume 8 provide regardi...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 states that clinical work conducted o...,single_hop_specifc_query_synthesizer
3,How does Title IV regulation define and handle...,[Non-Term Characteristics A program that measu...,"According to the context, Title IV programs ge...",single_hop_specifc_query_synthesizer
4,H0w do the stAndards for nonstandard terms app...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The standards for nonstandard terms define tha...,multi_hop_abstract_query_synthesizer
5,How do overlapping courses across terms relate...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Overlapping courses across terms are considere...,multi_hop_abstract_query_synthesizer
6,How do the disbursement timing requirements fo...,[<1-hop>\n\nboth the credit or clock hours and...,Disbursement timing for federal financial aid ...,multi_hop_abstract_query_synthesizer
7,Guidance and principles for determining disbur...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that disbursement timing ...,multi_hop_abstract_query_synthesizer
8,How does the inclusion of clinical work in sta...,[<1-hop>\n\nboth the credit or clock hours and...,The inclusion of clinical work in standard ter...,multi_hop_specific_query_synthesizer
9,How does Volume 8 explain the impact of accele...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 details that in clock-hour and non-te...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans on behalf of dependent students)  \n- Federal Stafford Loans (Subsidized and Unsubsidized)  \n- Federal SLS Loans  \n- Federal PLUS Loans\n\nNote that Federal Stafford, SLS, and PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before new loan authority ended on July 1, 2010, so no new loans are made under the FFEL Program after that date. Currently, Direct Loans (Subsidized, Unsubsidized, and PLUS) are the available loan types.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'complicated-picture-71' at:
https://smith.langchain.com/o/52c4cf8c-3e10-4738-ae6a-bc186d787252/datasets/259df13e-e770-402b-b13d-ccd2df029d6e/compare?selectedSessions=3f81ce78-25e6-4f8f-b180-5466a6769af3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 info about academic year...,I don't know.,None,Volume 2 explains that the academic year must ...,0,0,0,1.006592,6617efc7-1e72-43aa-bea3-d5235a6b8268,5fa8d9bd-7005-4f69-ae8d-3f23a52086e1
1,How does Chapter 3 address the inclusion of cl...,I don't know.,None,Chapter 3 provides guidance on including clini...,0,0,0,0.835988,84d02ec8-e28b-42d5-916a-43acf5e6cfa3,22a7a0f4-736a-44ad-9cbc-c93ca6fd22b3
2,How does Volume 8 explain the impact of accele...,"Based on the provided context, Volume 8 explai...",None,Volume 8 details that in clock-hour and non-te...,0,0,0,10.665155,1e060e49-8291-403a-96bf-a4e7992b666b,b026a8dc-1006-4bd1-bf9e-6d077a806d10
3,How does the inclusion of clinical work in sta...,Based on the provided context from Volume 8 an...,None,The inclusion of clinical work in standard ter...,1,1,0,9.098388,dfadba7a-f44a-437d-ab75-98483ad81287,ef4bf929-b144-4edf-a6cf-80295b59c360
4,Guidance and principles for determining disbur...,"Based on the provided context, guidance and pr...",None,The context explains that disbursement timing ...,1,1,0,5.322507,1b725df0-4e4d-4b87-9b1f-289024942bcc,c6b160d3-d1a0-41d6-a97c-efcd3d385eae
5,How do the disbursement timing requirements fo...,"Based on the provided context, the disbursemen...",None,Disbursement timing for federal financial aid ...,1,0,0,7.364672,91384139-39aa-4e3c-a8ce-77867d36a0f0,fb47aa6f-1040-4d37-8ee5-60c8421f4903
6,How do overlapping courses across terms relate...,Based on the provided context:\n\n- In general...,None,Overlapping courses across terms are considere...,1,1,0,9.842853,5a2af026-78d6-43df-9370-020ba6849221,c529d1ee-2e42-40f5-baa2-22d251370fde
7,H0w do the stAndards for nonstandard terms app...,Based on the provided context:\n\nNonstandard ...,None,The standards for nonstandard terms define tha...,1,1,0,11.992790,6e1944ee-9e42-4a8c-9ea5-4804a240cddc,2dc6ed3f-03a1-4130-913b-79d98df4e1c5
8,How does Title IV regulation define and handle...,"Based on the provided context, Title IV regula...",None,"According to the context, Title IV programs ge...",1,1,0,7.608856,5bf8fad8-5200-41eb-aa1d-32e144564e93,5f64c09c-0c50-4e8f-a23f-1f7337a72904
9,What information does Volume 8 provide regardi...,"Volume 8, specifically Chapter 3, provides add...",None,Volume 8 states that clinical work conducted o...,1,0,0,1.253426,95ad1ce3-d267-4cc5-87a3-75b4bd7276df,d90d2ed6-7d7c-4892-aed1-31e952351d29


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the context provided, it seems that there are a few different types of federal student loans available through the Direct Loan program:\n\n1. **Direct Subsidized Loans**: These loans are available based on the student's financial need, and the government pays the interest while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans**: These loans are available regardless of financial need, and interest accrues during all periods.\n\n3. **Direct PLUS Loans**: These are loans that parents of dependent students can take out to help cover the student’s cost of attendance, assuming they meet eligibility requirements. If the parent is ineligible, the student may become eligible for additional unsubsidized loans.\n\nAlso, for new borrowers, there are specific guidelines and entrance counseling requirements.\n\nI hope this helps clarify the kinds of loans available to you or your student. If you're feeling overwhelmed or have more 

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'earnest-sink-7' at:
https://smith.langchain.com/o/52c4cf8c-3e10-4738-ae6a-bc186d787252/datasets/259df13e-e770-402b-b13d-ccd2df029d6e/compare?selectedSessions=981a2c7b-812d-4af4-9927-d9e5e6e817cf




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 info about academic year...,Thank you for your question. I understand you'...,None,Volume 2 explains that the academic year must ...,0,0,1,4.656291,6617efc7-1e72-43aa-bea3-d5235a6b8268,27bbc94f-9f15-425a-80e1-21ebd646fad1
1,How does Chapter 3 address the inclusion of cl...,Thank you for your thoughtful question. Based ...,None,Chapter 3 provides guidance on including clini...,1,1,1,7.008431,84d02ec8-e28b-42d5-916a-43acf5e6cfa3,c98c417a-57b1-4134-be7e-8cbd7dfc653a
2,How does Volume 8 explain the impact of accele...,Thank you for your thoughtful question. I unde...,None,Volume 8 details that in clock-hour and non-te...,1,0,1,6.340113,1e060e49-8291-403a-96bf-a4e7992b666b,7141f629-e1f0-4710-8e54-c1ac2d77aa79
3,How does the inclusion of clinical work in sta...,Thank you for your thoughtful question. It sou...,None,The inclusion of clinical work in standard ter...,1,1,1,7.474088,dfadba7a-f44a-437d-ab75-98483ad81287,a0a08f4d-4534-4a95-b67a-60d63b464506
4,Guidance and principles for determining disbur...,Thank you for your thoughtful question. Based ...,None,The context explains that disbursement timing ...,1,1,1,4.638211,1b725df0-4e4d-4b87-9b1f-289024942bcc,ee37050c-546e-413d-97a8-b6fa4971b262
5,How do the disbursement timing requirements fo...,Thank you for your thoughtful question! I can ...,None,Disbursement timing for federal financial aid ...,1,1,1,8.888621,91384139-39aa-4e3c-a8ce-77867d36a0f0,4092cef0-ec4d-4b55-9ec6-9c0bbf2496fc
6,How do overlapping courses across terms relate...,Thank you for your thoughtful question. I unde...,None,Overlapping courses across terms are considere...,1,1,1,6.555660,5a2af026-78d6-43df-9370-020ba6849221,6a9b896f-13bb-421d-8126-500be52fffd8
7,H0w do the stAndards for nonstandard terms app...,Thank you for your thoughtful question; I can ...,None,The standards for nonstandard terms define tha...,1,1,1,8.582239,6e1944ee-9e42-4a8c-9ea5-4804a240cddc,112f50bf-caa2-490e-9c21-2b03266d7d45
8,How does Title IV regulation define and handle...,Thank you for your thoughtful question about h...,None,"According to the context, Title IV programs ge...",1,1,1,8.166677,5bf8fad8-5200-41eb-aa1d-32e144564e93,e14233f5-bf58-4de3-83c1-05e52b1eba2f
9,What information does Volume 8 provide regardi...,Thank you for your thoughtful question. Based ...,None,Volume 8 states that clinical work conducted o...,0,0,1,2.582215,95ad1ce3-d267-4cc5-87a3-75b4bd7276df,a34a01c2-86c5-4694-b4b8-c17b6757c3ed


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.